In [1]:
import os
os.chdir('..')

In [2]:
import pandas as pd
import numpy as np

In [3]:
!pwd

/Users/krishna/Documents/Research/Simultaneous Classifier/folktables-experiments/multidecomp


In [12]:
from bilevel.build_all_models import *
from bilevel.OnlineLinearExpert import OnlineLinearExpert
from bilevel.OnlineTreeExpert import OnlineHoeffdingTree
from bilevel.utils import ordinal_encoder

## one hot and ridge implementable validate

In [7]:
A_t = np.load('./data_frames/nparrays/acsog_allstates2021_20230808-095906_unshuffled__undropped_.npy')

In [8]:
df_oh = pd.read_pickle('./data_frames/dense_acs_mm_oh.pkl')

In [9]:
df_oh.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1569188 entries, 0 to 1630166
Columns: 898 entries, WKHP to RAC1P_9
dtypes: float64(3), uint8(895)
memory usage: 1.4 GB


In [10]:
df_oh

,WKHP,AGEP,PINCP,OCCP_10,OCCP_20,OCCP_40,OCCP_51,OCCP_52,OCCP_60,OCCP_101,...,MAR_5,RAC1P_1,RAC1P_2,RAC1P_3,RAC1P_4,RAC1P_5,RAC1P_6,RAC1P_7,RAC1P_8,RAC1P_9
0,0.295918,0.025316,0.039472,0,0,0,0,0,0,0,...,1,1,0,0,0,0,0,0,0,0
1,0.397959,0.037975,0.030967,0,0,0,0,0,0,0,...,1,0,1,0,0,0,0,0,0,0
2,0.173469,0.025316,0.030467,0,0,0,0,0,0,0,...,1,1,0,0,0,0,0,0,0,0
3,0.051020,0.215190,0.053479,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
4,0.091837,0.025316,0.009455,0,0,0,0,0,0,0,...,1,1,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1630162,0.397959,0.291139,0.399670,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,1
1630163,0.397959,0.379747,0.299615,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
1630164,0.397959,0.367089,0.092501,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
1630165,0.295918,0.063291,0.060483,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,1


In [11]:
y_dat = df_oh['PINCP']

In [12]:
df_oh.drop('PINCP', axis=1, inplace=True)

In [13]:
X_dat = df_oh

In [25]:
expridge = OnlineLinearExpert(X_dat, y_dat, l2_pen=1.0)

In [26]:
dir_name = './restruc_res/models/baseline'
filename =  'ridge_acsog'

In [28]:
b_ridgebase = build_baseline_alwayson(dir_name, filename, A_t, expridge)

100%|████████████████████████████████████████████████████████████████████████████████████████████████| 1569188/1569188 [23:07<00:00, 1131.00it/s]


In [18]:
len(b_ridgebase.expert.cumloss_groupwise)

12

In [31]:
import joblib
oridge_imp = joblib.load('./restruc_res/models/baseline/ridge_acsog20230822-001825.pkl')

In [32]:
for gnum in range(12):
  print(oridge_imp.cumloss_groupwise[gnum][-1])

31098.672186126223
22396.608111110658
37590.35674739708
3350.0869638965455
334.9671698399062
20.015414809307583
56.00525708187821
4901.493675804962
75.07590235612507
2203.9094416215803
4963.369724429311
53495.28029723699


In [35]:
y_dat[1]

0.03096703186752714

## build Anh validate linear

In [13]:
A_t = np.load('./data_frames/nparrays/acsog_allstates2021_20230808-095906_unshuffled__undropped_.npy')

In [14]:
A_t.shape

(1569188, 12)

In [15]:
df_oh = pd.read_pickle('./data_frames/dense_acs_mm_oh.pkl')
y_dat = df_oh['PINCP']
X_dat = df_oh

In [16]:
experts = [OnlineLinearExpert(X_dat[:1000], y_dat[:1000], l2_pen=1.0) for _ in range(A_t.shape[1])]

In [17]:
dir_name = './restruc_res/models/Anh'
filename =  'ridge_validate_acsog'

In [18]:
b_Anh = build_Anh(dir_name, filename, A_t[:1000], experts)

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1000/1000 [00:03<00:00, 284.49it/s]


## hoeffding tree acs ordering

In [4]:
A_t = np.load('./data_frames/nparrays/acsog_allstates2021_20230808-095906_unshuffled__undropped_.npy')

In [5]:
df_notoh = pd.read_pickle("./data_frames/dense_acs_mm_notoh.pkl")
cat_cols_sig = ['OCCP', 'SCHL', 'ST', 'JWTRNS', 'DRAT', 'COW', 'SEX', \
       'RELSHIPP', 'POBP', 'ENG', 'MAR', 'RAC1P'] # significant features from the earlier analysis
numeric_cols = ['WKHP', 'AGEP', 'PINCP']
print(len(cat_cols_sig), len(numeric_cols))
df_notoh = ordinal_encoder(df_notoh, cat_cols_sig)

12 3


In [6]:
y_dat = df_notoh['PINCP']
df_notoh.drop('PINCP', axis=1, inplace=True)

In [7]:
X_datdict = df_notoh.to_dict(orient='records')

In [8]:
exphoefftree = OnlineHoeffdingTree(X_datdict, y_dat)

In [9]:
dir_name = './restruc_res/models/baseline'
filename =  'htree_acsog'

In [10]:
b_htreebase = build_baseline_alwayson(dir_name, filename, A_t, exphoefftree)

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1569188/1569188 [03:04<00:00, 8514.80it/s]


In [11]:
import joblib
ohtree_imp = joblib.load('./restruc_res/models/baseline/htree_acsog20230822-010309.pkl')

In [12]:
for gnum in range(12):
  print(ohtree_imp.cumloss_groupwise[gnum][-1])

20745.077407449095
13810.410604688319
24103.089793125517
2402.4420325015653
237.78001319990227
14.85627330175781
36.72450443720022
3012.412629404053
54.10512373150721
1460.7881079631447
3233.2895344743274
34555.48801213991


In [19]:
y_dat.iloc[616]

0.041222672469858425